In [3]:
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from ecit import *


def ecit_simu(n_list=[400,800,2000,4000],
              t=100,
              cit=kcit,
              ensembles=[(p_alpha175,1), (p_alpha175,2), (p_alpha175,400)],
              show_progress=True,
              dz=1,n_indp='N',z_dis='gaussian',noise_dis='gaussian',noise_std=0.3,fun1=None,fun2=None):
    
    results = []

    for p_ensemble, k in ensembles:

        if show_progress: print(f"Ensemble (k={k}, {p_ensemble.__name__})") if k>1 else print(f"Default (k=1)")

        eI = [0]*len(n_list)
        eII = [0]*len(n_list)
        tim = [0]*len(n_list)

        for i, n in enumerate(n_list):

            start_time = time.time()
            iteration_loop = tqdm(range(t), desc=f"n={n}", leave=True, dynamic_ncols=True) if show_progress else range(t)
            for sed in iteration_loop:
                np.random.seed(sed)
                random.seed(sed)
                dataI = np.hstack((generate_samples(n=n,dz=dz,indp='C',z_dis=z_dis, noise_dis=noise_dis,noise_std=noise_std,
                                                    fun1=fun1, fun2=fun1)))
                dataII = np.hstack((generate_samples(n=n,dz=dz,indp=n_indp,z_dis=z_dis, noise_dis=noise_dis,noise_std=noise_std,
                                                     fun1=fun1, fun2=fun1)))
                obj_ECIT = ECIT(dataI, cit, p_ensemble, k if k<100 else int(n/k))
                pI = obj_ECIT([0], [1], [2])
                obj_ECIT = ECIT(dataII, cit, p_ensemble, k if k<100 else int(n/k))
                pII = obj_ECIT([0], [1], [2])
                if pI<0.05:
                    eI[i] += 1
                if pII>0.05:
                    eII[i] += 1

            end_time = time.time()
            tim[i] = end_time - start_time
            eI[i] = eI[i]/t
            eII[i] = eII[i]/t

        results.append([eI, eII, tim])

    return results




def show_results(results, ensembles, n_list):
    dir = {}
    for (p_ensemble, k), result in zip(ensembles, results):
        key_name = f"Ensemble (k={k}, {p_ensemble.__name__})" if k > 1 else f"Default (k=1)"
        dir[key_name] = result

    fig, axes = plt.subplots(2, 2, figsize=(16, 10), dpi=300, sharex=True)
    (ax_eI, ax_eII), (ax_eSum, ax_tim) = axes

    for label, result in dir.items():
        eI, eII, tim = result
        eSum = np.array(eI) + np.array(eII)

        ax_eI.plot(n_list, eI, label=label)
        ax_eII.plot(n_list, eII)
        ax_eSum.plot(n_list, eSum)
        ax_tim.plot(n_list, tim)

    ax_eI.set_title("Error I")
    ax_eI.set_ylabel("Error Rate")

    ax_eII.set_title("Error II")

    ax_eSum.set_title("Error I + Error II")
    ax_eSum.set_xlabel("Sample Size (n)")   
    ax_eSum.set_ylabel("Error Rate")


    ax_tim.set_title("Execution Time")
    ax_tim.set_ylabel("Time (s)")
    ax_tim.set_xlabel("Sample Size (n)")

    fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3, fontsize=12)

    ax_tim.set_xticks(n_list)
    ax_eSum.set_xticks(n_list)

    plt.tight_layout()
    plt.show()

In [12]:
n_list = [800, 1200, 2000]
ensembles = [(p_alpha2, 1), (p_alpha2, 400)]
t = 20
cit = kcit
results = ecit_simu(n_list=n_list, t=t, cit=cit, ensembles=ensembles,
                    dz=1,z_dis='gaussian',noise_dis='gaussian',noise_std=0.5,fun1=None,fun2=None)
show_results(results, ensembles, n_list)
results

Default (k=1)


n=1200:  50%|█████     | 10/20 [01:04<01:03,  6.32s/it]

| **method** | $\alpha$           |  $n$     |
|------------|--------------------|----------| 
|   KCIT     |         2          | 200, 400 |
|   CMIknn   |         1.25       | 500, 1000| 
|   CCIT     |         1.75       | 1000     |  

indp: 'N'

dz: 1-2-5

z_dis: 'gaussian'

noise_dis: 'gaussian'

noise_std: 1



| **N**  | **Category**        | **dz=1**           |             |             | **dz=2**           |             |             | **dz=5**           |             |             |
|--------|---------------------|--------------------|-------------|-------------|--------------------|-------------|-------------|--------------------|-------------|-------------|
|        |                     | **Type I**         | **Type II** | **Overall** | **Type I**         | **Type II** | **Overall** | **Type I**         | **Type II** | **Overall** |
| 800    |  KCIT                |                    |             |             |                    |             |             |                    |             |             |
|        | Ensemble (n=400)    |                    |             |             |                    |             |             |                    |             |             |
| 1200    | KCIT                |                    |             |             |                    |             |             |                    |             |             |
|        | Ensemble (n=400)    |                    |             |             |                    |             |             |                    |             |             |
| 2000   | KCIT                |                    |             |             |                    |             |             |                    |             |             |
|        | Ensemble (n=400)    |                    |             |             |                    |             |             |                    |             |             |
